In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")

print("Dataset baixado em:", path)

Using Colab cache for faster access to the 'cbis-ddsm-breast-cancer-image-dataset' dataset.
Dataset baixado em: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset


In [10]:
from google.colab import drive
import os

# Monta o Google Drive na máquina virtual do Colab
drive.mount('/content/drive')

# Verifica se o arquivo está lá (ajuste o caminho se mudou o nome da pasta)
zip_path = '/content/drive/MyDrive/Unifesp/Projeto_Cancer/archive.zip'

if os.path.exists(zip_path):
    print("Arquivo zip encontrado com sucesso!")
else:
    print(f"ERRO: Não encontrei o arquivo em {zip_path}. Verifique o nome da pasta/arquivo no Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo zip encontrado com sucesso!


In [13]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np
import cv2
import pywt
from skimage.restoration import wiener
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# --- 1. CONFIGURAÇÃO E EXTRAÇÃO ---
# Caminho exato identificado na sua imagem
ZIP_PATH = '/content/drive/MyDrive/Unifesp/Projeto_Cancer/archive.zip'
EXTRACT_PATH = '/content/dados_extraidos'

if not os.path.exists(EXTRACT_PATH):
    print(f"Descompactando {ZIP_PATH}...")
    try:
        with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_PATH)
        print("Descompactação concluída.")
    except Exception as e:
        print(f"Erro ao descompactar: {e}")
        exit() # Para se der erro
else:
    print("Arquivos já descompactados anteriormente.")

# --- 2. MAPEAMENTO E PREPARAÇÃO DO DATASET ---
print("Localizando e processando metadados (CSVs)...")

# Localiza os CSVs de descrição de massa (Tumores) recursivamente
csv_train = glob.glob(os.path.join(EXTRACT_PATH, "**", "mass_case_description_train_set.csv"), recursive=True)
csv_test = glob.glob(os.path.join(EXTRACT_PATH, "**", "mass_case_description_test_set.csv"), recursive=True)

if not csv_train:
    raise FileNotFoundError("CSV de treino não encontrado dentro do zip.")

# Carrega e unifica
df = pd.read_csv(csv_train[0])
if csv_test:
    df_test = pd.read_csv(csv_test[0])
    df = pd.concat([df, df_test], ignore_index=True)

# Mapeamento de Classes conforme o Artigo [cite: 222-224]
# Normal (Benign without callback), Benign, Malignant
def get_article_label(pathology):
    if 'MALIGNANT' in pathology: return 'Malignant'
    if 'BENIGN_WITHOUT_CALLBACK' in pathology: return 'Normal'
    if 'BENIGN' in pathology: return 'Benign'
    return None

df['label'] = df['pathology'].apply(get_article_label)
df = df.dropna(subset=['label'])

# Indexação das Imagens Físicas (.jpg)
print("Indexando imagens físicas no disco...")
image_map = {}
for img_path in glob.glob(os.path.join(EXTRACT_PATH, "**", "*.jp*g"), recursive=True):
    image_map[os.path.basename(img_path)] = img_path

# Cruzamento CSV <-> Imagem
valid_data = []
for _, row in df.iterrows():
    # O ID base da imagem está na primeira parte do caminho no CSV
    base_id = row['image file path'].split('/')[0].strip()

    found_path = None
    # Prioridade 1: Imagem CROP (Recorte do tumor - ROI)
    for fname in image_map:
        if base_id in fname and "CROP" in fname:
            found_path = image_map[fname]
            break
    # Prioridade 2: Imagem FULL (Mamografia completa) se não houver crop
    if not found_path:
        for fname in image_map:
            if base_id in fname and "FULL" in fname:
                found_path = image_map[fname]
                break

    if found_path:
        valid_data.append({'filepath': found_path, 'label': row['label']})

df_final = pd.DataFrame(valid_data)
print(f"Total de imagens vinculadas prontas para uso: {len(df_final)}")
print("Distribuição:", df_final['label'].value_counts())

# Divisão Treino/Teste (70/30) [cite: 258, 324]
train_df, val_df = train_test_split(df_final, test_size=0.3, stratify=df_final['label'], random_state=42)

# --- 3. PIPELINE DE PRÉ-PROCESSAMENTO (Fig. 1 do Artigo) ---
def process_pipeline(image_path):
    # Leitura
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None: return np.zeros((224,224,3))

    # Redimensionar para entrada da GoogleNet [cite: 152]
    img = cv2.resize(img, (224, 224))

    # A) Otsu Thresholding (Remoção de Fundo/Músculo) [cite: 84-86]
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    img = cv2.bitwise_and(img, img, mask=mask)

    # B) Wiener Filter (Remoção de Ruído, SNR ~0.2) [cite: 87, 97-105]
    img = img.astype(np.float64) / 255.0
    psf = np.ones((5, 5)) / 25
    img = wiener(img, psf, balance=0.2)
    img = np.clip(img * 255, 0, 255).astype(np.uint8)

    # C) CLAHE Filter (Realce de Contraste) [cite: 106-108]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)

    # D) Wavelet Packet Decomposition (db3, nível 2) para Suavização [cite: 109-115]
    # Decompõe
    coeffs = pywt.wavedec2(img, 'db3', level=2)
    # Zera os coeficientes de detalhe do nível 1 para suavizar (Denoising)
    coeffs[1] = tuple([np.zeros_like(v) for v in coeffs[1]])
    # Reconstrói
    img = pywt.waverec2(coeffs, 'db3')
    img = np.clip(img, 0, 255).astype(np.uint8)

    # Converter para RGB (GoogleNet exige 3 canais)
    return cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) / 255.0

# Gerador de Dados (Para processar em tempo real sem estourar a RAM)
class DDSMGenerator(Sequence):
    def __init__(self, df, batch_size=16): # Batch ajustado
        self.df = df
        self.batch_size = batch_size
        self.le = LabelEncoder()
        self.df['enc'] = self.le.fit_transform(self.df['label'])
        self.n_classes = 3
        self.indices = np.arange(len(self.df))

    def __len__(self): return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_data = self.df.iloc[indices]
        X, y = [], []
        for _, row in batch_data.iterrows():
            X.append(process_pipeline(row['filepath']))
            y.append(row['enc'])
        return np.array(X), to_categorical(y, num_classes=3)

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

train_gen = DDSMGenerator(train_df, batch_size=10) # Batch 10 conforme [cite: 326]
val_gen = DDSMGenerator(val_df, batch_size=10)

# --- 4. REDE NEURAL E TREINAMENTO (ALTERAÇÃO DE PESOS) ---
print("\nInicializando GoogleNet com Adam...")

# GoogleNet (Inception) [cite: 14, 260]
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Camadas Finais Modificadas [cite: 279]
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x) # Dropout 50%
x = Dense(1024, activation='relu')(x) # Fully Connected
predictions = Dense(3, activation='softmax')(x) # 3 Classes

model = Model(inputs=base_model.input, outputs=predictions)

# Compilação: Adam, LR=0.0001 (O melhor cenário do artigo) [cite: 15, 536, 618]
# Aqui ocorre a definição de como os pesos serão alterados
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Iniciando treinamento por 5 épocas (Demonstração da alteração de pesos)...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5, # Use 10 ou mais para replicar totalmente, 5 para teste rápido
    verbose=1
)

print("Treinamento concluído.")

Descompactando /content/drive/MyDrive/Unifesp/Projeto_Cancer/archive.zip...
Descompactação concluída.
Localizando e processando metadados (CSVs)...
Indexando imagens físicas no disco...
Total de imagens vinculadas prontas para uso: 0


KeyError: 'label'